In [2]:
import transformers
import json

import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets

import pandas as pd
import json
import numpy as np
covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'

def get_data_from_json(filename):
    jsonfile = open(covid_file, 'r')
    data = jsonfile.read()
    jsonfile.close()
    return json.loads(data)

covid_data = get_data_from_json(covid_file)
bio_data = get_data_from_json(bio_file)

#datasets.set_caching_enabled(False)

Reusing dataset squad (/soe/jafidler/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)


In [3]:
from collections import OrderedDict
def make_and_save_full_dataset(covid=None, squad = None, bioASQ = None, path = '../data/squad_bioASQ_covidQA/'):
    squad = datasets.Dataset.from_dict(squad_qa[:])
    bioASQ = datasets.Dataset.from_dict(bioASQ[:])
    if covid is not None:
        full_data = datasets.dataset_dict.DatasetDict({'squad':squad, 'covid':covid,  'bio':bioASQ})
    else:
        full_data = datasets.dataset_dict.DatasetDict({'squad':squad,'bio':bioASQ})

    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

In [4]:
covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)
squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])

covid_bio_squad_dataset_path = "../data/squad_bioASQ_covidQA/"

#this is just for testing purposes, I am going to make both of these files very small only at max 3000 datasets
# squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
# bio_qa = datasets.Dataset.from_dict(bio_qa[:20])
# covid_qa = datasets.Dataset.from_dict(covid_qa[:10])

Using custom data configuration default-1d18a620853d8414


LOADING DATAFILES
{'train': '../data/COVID-QA.json'}


0 examples [00:00, ? examples/s]

Using custom data configuration default-d665e54172161a2c


Dataset squad downloaded and prepared to /soe/jafidler/.cache/huggingface/datasets/squad/default-1d18a620853d8414/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e. Subsequent calls will reuse this data.
LOADING DATAFILES
{'train': '../bioASQ/bioASQ.json'}


0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /soe/jafidler/.cache/huggingface/datasets/squad/default-d665e54172161a2c/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e. Subsequent calls will reuse this data.


In [14]:
make_and_save_full_dataset(squad=squad_qa,bioASQ=bio_qa,path=covid_bio_squad_dataset_path)

K = 8
to_remove_per_step = int(squad_qa.num_rows / K)
bio_remove_per_step = int(bio_qa.num_rows / K)

In [21]:
def run_gradual_ft(output_dir, checkpoint, k_fold):
    !CUDA_VISIBLE_DEVICES=2 python run_qa.py \
      --model_name_or_path {checkpoint} \
      --dataset_name ../data/squad_bioASQ_covidQA/\
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 32\
      --per_device_eval_batch_size 32\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 8e-6 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --k_fold_cross_valid {k_fold} \
      --output_dir {output_dir} \
      --overwrite_output_dir


In [ ]:
k_fold = 5
directory = '../models/gradual_ft_baseline2/'
output_dir = '../models/gradual_ft_baseline2/Split-' + str(k_fold)+'/'

squad_total_len = squad_qa.num_rows
bio_total_len = bio_qa.num_rows
import sys
log_file = open('log_file.txt','w')
sys.stdout = log_file

for i in range(K):
    print('\n\n**************************************************')
    print('==================================================')
    print('          At Gradual Fine Tuning Step: ',i+1)
    print('**************************************************')
    print('==================================================\n\n')
    if i < 1:
        run_gradual_ft(directory,'roberta-base', k_fold )
    else:
        run_gradual_ft(directory, output_dir, k_fold)
        squad_qa.shuffle()
        bio_qa.shuffle()
        covid_qa = datasets.Dataset.from_dict(covid_qa[:])

    if i < 4: #remove from squad
        if to_remove_per_step > squad_qa.num_rows:
            print('not enough data')
            break
        print(f'\n========== Removing {to_remove_per_step} from SQuAD ({squad_qa.num_rows}/{squad_total_len})==========')
        squad_qa = datasets.Dataset.from_dict(squad_qa[:-to_remove_per_step])
        bio_temp = datasets.Dataset.from_dict(bio_qa[:1])
        make_and_save_full_dataset(squad=squad_qa, bioASQ=bio_temp, path=covid_bio_squad_dataset_path)
    else: #remove from bioASQ
        if to_remove_per_step > bio_qa.num_rows:
            print('not enough data')
            break
        print(f'\n========= Removing {bio_remove_per_step} from BioASQ ({bio_qa.num_rows}/{bio_total_len}) =========')
        bio_qa = datasets.Dataset.from_dict(bio_qa[:-bio_remove_per_step])
        squad_temp = datasets.Dataset.from_dict(squad_qa[:1])
        make_and_save_full_dataset(squad=squad_temp, bioASQ=bio_qa,path=covid_bio_squad_dataset_path)

print('Finished process')
sys.stdout = sys.__stdout__
log_file.close()